In [1]:
%pip install pdfplumber
%pip install groq
%pip install --upgrade tiktoken
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 8.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.6.0
    Uninstalling tiktoken-0.6.0:
      Successfully uninstalled tiktoken-0.6.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [110]:
import pdfplumber

extracted_text = []
extracted_tables = []
extracted_context = []

def check_bboxes(word, table_bbox):
    """
    Check whether word is inside a table bbox.
    """
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

with pdfplumber.open("./inputs/sample-1-5.pdf") as pdf:
    for pg_number in range(len(pdf.pages)):
        page = pdf.pages[pg_number]
        tables = page.find_tables()
        table_bboxes = [i.bbox for i in tables]
        tables = [{'table': i.extract(), 'doctop': i.bbox[1]} for i in tables]
        non_table_words = [word for word in page.extract_words() if not any(
            [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
            
        if len(tables) > 0 and len(non_table_words) > 0:
            extracted_tables += tables
            extracted_text.append(' '.join([i['text'] for i in non_table_words])) 
            extracted_context.append({"pg_number" :pg_number  , "text" :  [' '.join([i['text'] for i in non_table_words])] , "tables" : tables })
        elif len(tables) > 0:
            extracted_tables += tables
            extracted_context.append({"pg_number" :pg_number  , "text" : "none" , "tables" : tables })

        elif len(non_table_words) > 0:
            extracted_text.append(' '.join([i['text'] for i in non_table_words])) 
            extracted_context.append({"pg_number" :pg_number  , "text" : [' '.join([i['text'] for i in non_table_words])]  , "tables" : "none" })
            


In [3]:

required_schema = {
    "type": "object",
    "properties": {
      "company_name": {
        "type": "string",
        "description": "Name of the company to which pdf belong to"
      },
      "data": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "lifetime_maximum": {
              "type": "string",
              "description": "In-Network value of lifetime_maximum"
            },
            "plan_coinsurance": {
              "type": "number",
              "description": "In-Network value of plan_coinsurance"
            },
            "physician_services_primary_care": {
              "type": "string",
              "description": "In-Network value of Physician Services Primary Care Physician (PCP) Services/Office Visit"
            },
            "virtual_physician_primary_care": {
              "type": "string",
              "description": "In-Network value of Virtual Physician Primary Care Physician (PCP) Services/Office Visit"
            },
            "advanced_radiological": {
              "type": "string",
              "description": "In-Network value of Advanced Radiological Imaging (ARI)"
            },
            "outpatient_outpatient_facility": {
              "type": "string",
              "description": "In-Network value of Outpatient Outpatient Facility"
            },
            "hospice_inpatient_facilities": {
              "type": "string",
              "description": "In-Network value of Hospice Inpatient Facilities"
            }
          },
          "required": [
            "lifetime_maximum",
            "plan_coinsurance",
            "physician_services_primary_care",
            "virtual_physician_primary_care",
            "outpatient_outpatient_facility",
            "hospice_inpatient_facilities"
          ]
        }
      }
    },
    "required": ["company_name", "data"]
  }

In [146]:
from groq import Groq

client = Groq(api_key="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")

def get_model_response(system_prompt, current_page_context, previous_output, schema, model_name):
    # print(previous_output, schema, model_name)
    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": "Previous Context: " + str(previous_output)
        },
        {
            "role": "user",
            "content": "Current Context: " + str(current_page_context)
        },
        {
            "role": "user",
            "content": "Compare current context with previous context and Generate accurate value for required schema. Note some key value are not present in current context / previous context. If value not present dont return in json output"
        },
        {
            "role": "user",
            "content": "output : dont return the json key value pair which do not have value in above context"
        },
        
    ]
    
    if schema:
        messages.append({
            "role": "user",
            "content": "Schema: " + str(schema)
        })

    chat_completion = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=0,
        max_tokens=8192,
        top_p=1,
        stream=False,
        response_format={"type": "json_object"},
        stop=None,
    )

    return chat_completion


In [150]:
import time
system_prompt = "you are a HelpFull Json Generator from the given context based on the provided schema. ignore the key value pair which are not present in the context."
op = {
"company_name": "Luna Gourmet Coffee & Tea Company, LLC",
"data": [
{
"lifetime_maximum": "Unlimited",
"plan_coinsurance": 0.8
}
]
}

previous_context = get_model_response(system_prompt,extracted_context[0] ,"", required_schema , "llama3-70B-8192")
previous_context = previous_context.choices[0].message.content
print("page_no : " , "0" ,previous_context)
res = []

for page_context in extracted_context[1:]:
    response = get_model_response(system_prompt, page_context, previous_context, required_schema , "llama3-70B-8192")
    res.append(response.choices[0].message.content)
    previous_context = response.choices[0].message.content
    print("page_no : " , page_context['pg_number'] ,previous_context)
    time.sleep(6)




page_no :  0 {
"company_name": "Luna Gourmet Coffee & Tea Company, LLC",
"data": [
{
"lifetime_maximum": "Unlimited",
"plan_coinsurance": 0.8
}
]
}
page_no :  1 {
"company_name": "Luna Gourmet Coffee & Tea Company, LLC",
"data": [
{
"lifetime_maximum": "Unlimited",
"plan_coinsurance": 0.8,
"physician_services_primary_care": "Plan pays 80% ^",
"virtual_physician_primary_care": "Plan pays 80% ^"
}
]
}
page_no :  2 {
"company_name": "Luna Gourmet Coffee & Tea Company, LLC",
"data": [
{
"lifetime_maximum": "Unlimited",
"plan_coinsurance": 0.8,
"physician_services_primary_care": "Plan pays 80% ^",
"virtual_physician_primary_care": "Plan pays 80% ^"
}
]
}
page_no :  3 {
"company_name": "Luna Gourmet Coffee & Tea Company, LLC",
"data": [
{
"lifetime_maximum": "Unlimited",
"plan_coinsurance": 0.8,
"physician_services_primary_care": "Plan pays 80% ^",
"virtual_physician_primary_care": "Plan pays 80% ^",
"outpatient_outpatient_facility": "Plan pays 80% ^",
"hospice_inpatient_facilities": "Plan p

In [151]:
print(previous_context)

{
"company_name": "Luna Gourmet Coffee & Tea Company, LLC",
"data": [
{
"lifetime_maximum": "Unlimited",
"plan_coinsurance": 0.8,
"physician_services_primary_care": "Plan pays 80% ^",
"virtual_physician_primary_care": "Plan pays 80% ^",
"outpatient_outpatient_facility": "Plan pays 80% ^",
"hospice_inpatient_facilities": "Plan pays 80% ^"
}
]
}
